In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Intellectual_Disability/GSE200864'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Definition and prognostic impact of Ph-like and IKZF1plus features in children with Down Syndrome Acute Lymphoblastic Leukaemia"
!Series_summary	"Background"
!Series_summary	"Children with Down Syndrome have an augmented risk for B-cell acute lymphoblastic leukaemia (DS-ALL), which is associated with a survival lower than in non-DS ALL, due to increased chemotherapy-related toxicity and a higher relapse rate, thus demanding new tailored therapeutic strategies."
!Series_summary	"Cytogenetic abnormalities common in childhood ALL are less frequent in DS-ALL, while alterations in CRLF2 and IKZF1 genes are increased."
!Series_summary	"Aim of the study was to evaluate in DS-ALL children the incidence and prognostic value of the Philadelphia Chromosome-Like (Ph-like) status and the “IKZF1plus” profile, both associated with poor outcome in non-DS ALL and therefore introduced in current therapeutic protocols for BCP-ALL."
!Series_summary	"Method"
!Series_s

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determining if gene expression data is available
series_title = "Definition and prognostic impact of Ph-like and IKZF1plus features in children with Down Syndrome Acute Lymphoblastic Leukaemia"
series_summary = [
    "Background",
    "Children with Down Syndrome have an augmented risk for B-cell acute lymphoblastic leukaemia...",
    "Cytogenetic abnormalities common in childhood ALL are less frequent in DS-ALL...",
    "Gene expression was measured using Affymetrix platform in DownSyndrome BCP-ALL pediatric patients at diagnosis",
    # ... other summary lines
]

gene_expression_keywords = ["gene expression", "Affymetrix platform", "measured"]

# Check if gene expression data is present
if any(keyword in series_title for keyword in gene_expression_keywords) or \
   any(keyword in summary for summary in series_summary for keyword in gene_expression_keywords):
    is_gene_available = True

# Checking for the presence of the variables in the provided dictionary
sample_characteristics = {
    0: ['treatment: pt1', 'treatment: pt2', 'treatment: pt3', 'treatment: pt4', 'treatment: pt5', 'treatment: pt6', 'treatment: pt7', 'treatment: pt8', 'treatment: pt9', 'treatment: pt10', 'treatment: pt11', 'treatment: pt12', 'treatment: pt13', 'treatment: pt14', 'treatment: pt15', 'treatment: pt16', 'treatment: pt17', 'treatment: pt18', 'treatment: pt19', 'treatment: pt20', 'treatment: pt21', 'treatment: pt22', 'treatment: pt23', 'treatment: pt24', 'treatment: pt25', 'treatment: pt26', 'treatment: pt27', 'treatment: pt28', 'treatment: pt29', 'treatment: pt30'],
    1: ['treatment: BCP-ALL at diagnosis'],
    2: ['tissue: Bone Marrow'],
}

trait_row = None  # No specific key for 'Intellectual_Disability'
age_row = None    # No specific key for 'age'
gender_row = None # No specific key for 'gender'

# Define the conversion functions
def convert_trait(value):
    try:
        val = value.split(":")[1].strip().lower()
        if val in ['yes', '1']:
            return 1
        elif val in ['no', '0']:
            return 0
    except:
        pass
    return None  # Unavailable or unknown values

def convert_age(value):
    try:
        val = float(value.split(":")[1].strip())
        return val
    except:
        return None  # Unavailable or unknown values

def convert_gender(value):
    try:
        val = value.split(":")[1].strip().lower()
        if val == 'male':
            return 1
        elif val == 'female':
            return 0
    except:
        pass
    return None  # Unavailable or unknown values

# Save cohort information
save_cohort_info('GSE200864', './preprocessed/Intellectual_Disability/cohort_info.json', is_gene_available, trait_row is not None)
